## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time 
from datetime import datetime
import os 
#from citipy import citipy
import requests
import gmaps

from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,45.48,79,64,8.57,broken clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.56,83,20,6.91,few clouds
2,2,Flinders,AU,-34.5833,150.8552,54.18,66,52,7.00,broken clouds
3,3,Airai,TL,-8.9266,125.4092,53.38,85,35,2.55,scattered clouds
4,4,Shingu,JP,33.7333,135.9833,74.43,87,100,4.90,overcast clouds


In [11]:
# 2. Prompt the 
#0user to enter minimum and maximum temperature criteria 
try:
    maX= int(input("What would you like the maximum temperature to be?"))
    miN= int(input("What would you like the minimum temperature to be?"))
except: 
    print("please enter only integers")


    

What would you like the maximum temperature to be?80
What would you like the minimum temperature to be?50


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
NeededData=pd.DataFrame(city_data_df.loc[(city_data_df["Max Temp"]<=maX)&(city_data_df["Max Temp"]>=miN)])
NeededData





,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,73.56,83,20,6.91,few clouds
2,2,Flinders,AU,-34.5833,150.8552,54.18,66,52,7.00,broken clouds
3,3,Airai,TL,-8.9266,125.4092,53.38,85,35,2.55,scattered clouds
4,4,Shingu,JP,33.7333,135.9833,74.43,87,100,4.90,overcast clouds
5,5,Busselton,AU,-33.6500,115.3333,53.35,93,100,13.11,light rain
...,...,...,...,...,...,...,...,...,...,...
683,683,Louga,RU,58.7372,29.8453,55.22,95,99,4.32,light rain
685,685,Oktyabrskiy,RU,54.4815,53.4710,67.60,81,100,1.83,overcast clouds
687,687,Tupanatinga,BR,-8.7533,-37.3397,70.50,74,73,14.23,light rain
688,688,Antalaha,MG,-14.9003,50.2788,68.02,86,62,9.71,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
NeededData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419 entries, 1 to 689
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           419 non-null    int64  
 1   City                 419 non-null    object 
 2   Country              412 non-null    object 
 3   Lat                  419 non-null    float64
 4   Lng                  419 non-null    float64
 5   Max Temp             419 non-null    float64
 6   Humidity             419 non-null    int64  
 7   Cloudiness           419 non-null    int64  
 8   Wind Speed           419 non-null    float64
 9   Current Description  419 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 36.0+ KB


In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#no null rows found 
NeededData=NeededData.dropna()


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = NeededData[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
2,Flinders,AU,54.18,broken clouds,-34.5833,150.8552,
3,Airai,TL,53.38,scattered clouds,-8.9266,125.4092,
4,Shingu,JP,74.43,overcast clouds,33.7333,135.9833,
5,Busselton,AU,53.35,light rain,-33.6500,115.3333,
7,Bredasdorp,ZA,56.95,clear sky,-34.5322,20.0403,
8,East London,ZA,71.73,clear sky,-33.0153,27.9116,
10,Mataura,NZ,51.48,broken clouds,-46.1927,168.8643,
12,Atuona,PF,77.74,few clouds,-9.8000,-139.0333,
14,Ketchikan,US,57.67,overcast clouds,55.3422,-131.6461,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.

record_count=1
set_count=1
hotel_list=[]
params = {
    "radius": 5000,
    "type":"lodging",
    "key": g_key,
}
# Use the lat/lng we recovered to identify airports
for i , row in hotel_df.iterrows():
    
    if (i % 100 == 0 and i >= 100):
        set_count += 1
        record_count = 1
        print("Cool Down 100 sec")
        time.sleep(100)
        
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    try: 
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotel = requests.get(base_url, params=params)
        hotel=hotel.json()
        hotel_=hotel["results"][0]["name"]
        hotel_list.append(hotel_)
        print((f"{hotel_}|Lat:{lat} Lng:{lng}"))
        
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    except: 
        hotel_list.append("NaN")
        print("NaN")
        pass
              
    record_count+=1
    print((f"set count:{set_count}| record count:{record_count}"))

print("--------------")
print("all finished")
print("--------------")

Keleti Beach Resort|Lat:-21.2 Lng:-175.2
set count:1| record count:2
Shellharbour Resort & Conference Centre|Lat:-34.5833 Lng:150.8552
set count:1| record count:3
NaN
set count:1| record count:4
Shingu UI Hotel|Lat:33.7333 Lng:135.9833
set count:1| record count:5
Observatory Guest House|Lat:-33.65 Lng:115.3333
set count:1| record count:6
Bredasdorp Country Manor|Lat:-34.5322 Lng:20.0403
set count:1| record count:7
Tu Casa|Lat:-33.0153 Lng:27.9116
set count:1| record count:8
Ellie's Villa|Lat:-46.1927 Lng:168.8643
set count:1| record count:9
Villa Enata|Lat:-9.8 Lng:-139.0333
set count:1| record count:10
Snow's Cove Lodge|Lat:55.3422 Lng:-131.6461
set count:1| record count:11
Kak Doma|Lat:50.6789 Lng:156.125
set count:1| record count:12
Okhotsk Palace Hotel|Lat:44.3525 Lng:143.3525
set count:1| record count:13
39 On Nile Guest House|Lat:-33.918 Lng:25.5701
set count:1| record count:14
Bluff Homestead - Guesthouse & Campervan Park|Lat:-46.6 Lng:168.3333
set count:1| record count:15
Groot

Antay Casino Hotel|Lat:-27.3667 Lng:-70.3333
set count:2| record count:49
The Fort Nelson Hotel|Lat:58.8053 Lng:-122.7002
set count:2| record count:50
Casa Laguna, Bed & Breakfast|Lat:27.9769 Lng:-114.0611
set count:2| record count:51
Le Rocher|Lat:-12.7887 Lng:45.2699
set count:2| record count:52
Hembre Gård|Lat:63.4644 Lng:11.114
set count:2| record count:53
NaN
set count:2| record count:54
Hotel Galvarino|Lat:-22.092 Lng:-70.1979
set count:2| record count:55
Shwe Pyi Soe Guest House, Thaton|Lat:16.9206 Lng:97.3714
set count:2| record count:56
NaN
set count:2| record count:57
Hotel Villa Kitzia Huacho|Lat:-11.0964 Lng:-77.6139
set count:2| record count:58
TDC Village (Hundred Man Camp)|Lat:-14.25 Lng:129.55
set count:2| record count:59
Lamana Hotel (Port Moresby)|Lat:-9.4431 Lng:147.1797
set count:2| record count:60
Ptarmigan Inn|Lat:60.8156 Lng:-115.7999
set count:2| record count:61
Nordkapp Vandrerhjem|Lat:70.9821 Lng:25.9704
set count:2| record count:62
Tanna Lodge|Lat:-19.55 Lng:

Floriana|Lat:50.751 Lng:33.4747
set count:4| record count:41
Mini-hotel «Quiet Harbor»|Lat:53.1406 Lng:140.73
set count:4| record count:42
Hotel Vision|Lat:-14.8031 Lng:36.5372
set count:4| record count:43
Sibles PG Inn|Lat:57.4035 Lng:45.0916
set count:4| record count:44
Hotel-Gasthof Zum Hirsch|Lat:47.7833 Lng:10.6167
set count:4| record count:45
Uyut|Lat:58.3814 Lng:97.4531
set count:4| record count:46
ROYAL PALACE HOTEL - JUBA|Lat:4.8517 Lng:31.5825
set count:4| record count:47
Skjervøy fiskecamp AS|Lat:70.0311 Lng:20.9714
set count:4| record count:48
Shimoda Tokyu Hotel|Lat:34.6667 Lng:138.95
set count:4| record count:49
Hostal Oro Negro|Lat:-4.5772 Lng:-81.2719
set count:4| record count:50
Mercure Ayr Hotel|Lat:55.4627 Lng:-4.6339
set count:4| record count:51
Chez Roger|Lat:-16.1667 Lng:49.7667
set count:4| record count:52
Veranda Paul et Virginie Hotel & Spa|Lat:-20.0064 Lng:57.6608
set count:4| record count:53
Cool Down 100 sec
Гостиница Домашняя|Lat:52.0 Lng:127.7
set count:5|

Mines Inn Hotel|Lat:4.8823 Lng:101.9644
set count:6| record count:46
Gostinitsa Kvartirnogo Tipa|Lat:47.6967 Lng:131.0981
set count:6| record count:47
Hotel Viktoria|Lat:69.4058 Lng:86.1778
set count:6| record count:48
Glamping & Camping|Lat:63.4427 Lng:-20.2734
set count:6| record count:49
Airbnb rooms|Lat:62.2266 Lng:-6.589
set count:6| record count:50
Westmark Fairbanks Hotel & Conference Center|Lat:64.8378 Lng:-147.7164
set count:6| record count:51
Sweet Sharons bnb|Lat:48.6499 Lng:-53.1147
set count:6| record count:52
Turkompleks Malyye Karely|Lat:64.4743 Lng:40.8524
set count:6| record count:53
NaN
set count:6| record count:54
NaN
set count:6| record count:55
Gostinitsa-Kafe, Alena|Lat:54.675 Lng:20.1347
set count:6| record count:56
departamento comodo|Lat:-36.7167 Lng:-73.1167
set count:6| record count:57
Gannan Supply and Marketing Cooperatives Hostel|Lat:47.9147 Lng:123.5031
set count:6| record count:58
"Plantation Bed and Breakfast|Lat:-21.3667 Lng:55.6167
set count:6| record

In [31]:
hotel_list

['Keleti Beach Resort',
 'Shellharbour Resort & Conference Centre',
 'NaN',
 'Shingu UI Hotel',
 'Observatory Guest House',
 'Bredasdorp Country Manor',
 'Tu Casa',
 "Ellie's Villa",
 'Villa Enata',
 "Snow's Cove Lodge",
 'Kak Doma',
 'Okhotsk Palace Hotel',
 '39 On Nile Guest House',
 'Bluff Homestead - Guesthouse & Campervan Park',
 'Groote Eylandt Lodge',
 'Shell Hotel - Xinzhou',
 "Don Eddie's Sport Fishing Center",
 'Gran Hotel Mar del Plata',
 'Scoop Guest House',
 'DoubleFAG',
 'Hostel At-Bashy',
 'Lunny Kot Guest House',
 'End of The Road Ltd.',
 'People ThankYou',
 'Nuevo Hotel Plaza',
 'Misty Waves Boutique Hotel',
 'Shanti Maurice Resort & Spa',
 'Sheraton Kauai Coconut Beach Resort',
 'Finch Bay Galapagos Hotel',
 'My Den Beachfront Bed and Breakfast | Self-Catering',
 'CHACARA BAILLY',
 'Casa Mika Ciobanus',
 'Sungurlu Ogretmenevi',
 'Sleep In Narsaq',
 'Hostal Las Lilas',
 'Auberge Le Caribou',
 'Hotel FX 72',
 'Inubosaki Kanko Hotel',
 'lounges and warm parking',
 'Avtov

In [32]:
hotel_df["Hotel Name"]=hotel_list

In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Flinders,AU,54.18,broken clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
3,Airai,TL,53.38,scattered clouds,-8.9266,125.4092,NaN
4,Shingu,JP,74.43,overcast clouds,33.7333,135.9833,Shingu UI Hotel
5,Busselton,AU,53.35,light rain,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
683,Louga,RU,55.22,light rain,58.7372,29.8453,U Lysoy Gory Hotel
685,Oktyabrskiy,RU,67.60,overcast clouds,54.4815,53.4710,Гостиница Октябрьская
687,Tupanatinga,BR,70.50,light rain,-8.7533,-37.3397,Pousada Central
688,Antalaha,MG,68.02,broken clouds,-14.9003,50.2788,Hotel Vitasoa


In [43]:
hotel_df.drop(hotel_df[hotel_df["Hotel Name"]=="NaN"].index, inplace=True)

In [44]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Flinders,AU,54.18,broken clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
4,Shingu,JP,74.43,overcast clouds,33.7333,135.9833,Shingu UI Hotel
5,Busselton,AU,53.35,light rain,-33.6500,115.3333,Observatory Guest House
7,Bredasdorp,ZA,56.95,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
...,...,...,...,...,...,...,...
683,Louga,RU,55.22,light rain,58.7372,29.8453,U Lysoy Gory Hotel
685,Oktyabrskiy,RU,67.60,overcast clouds,54.4815,53.4710,Гостиница Октябрьская
687,Tupanatinga,BR,70.50,light rain,-8.7533,-37.3397,Pousada Central
688,Antalaha,MG,68.02,broken clouds,-14.9003,50.2788,Hotel Vitasoa


In [45]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template =  """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))